# Algebraic graph theory
Algebraic graph theory is concerned with the study of graphs through several related matrices.
We then need to understand how arrays are represented and can be manipulated with Python using the library NumPy.

## Array creation
You can create a numpy array from a Python list or tuple using the  `array ` function

In [ ]:
import numpy as np
a = np.array([2,3,4])
print("a=", a)
print("Dimension of a:", a.shape)

 `array ` transforms lists of lists into two-dimensional arrays, i.e., matrices

In [ ]:
b = np.array([[1.5,2,3], [4,5,6]])
print(b)
print("Dimension of b:", b.shape)

The function  `zeros ` creates an array full of zeros, the function  `ones ` creates an array full of ones

In [ ]:
print(np.zeros((3, 4))) # tuple (3,4) is the shape of the zero array to be created
print(np.ones((2,3)))

One can create arrays whose elements space in a given range using  `arange `

In [ ]:
# the first two arguments are the starting point (included) and the ending point (excluded)
# the third argument is the step-size
np.arange(10, 30, 5) 

or using the function `linspace` that receives as an argument the number of elements that we want to obtain in the given range, instead of the step size

In [ ]:
# the first two arguments are the starting and ending point (both included!), 
# third argument is the number of elements
np.linspace(0, 2, 9 )

To summarize, numpy `ndarray` can represent arrays of any dimension but we will restrict to dimension 1 (vectors) and 2 (matrices). One-dimensional arrays are printed as rows, bidimensionals as matrices.

In [ ]:
# np.arange(): default start value is 0, default step size is 1
a = np.arange(6)                         # 1d array
print("a:", a, "\n")
b = np.arange(12).reshape(4,3)           # 2d array
print("b: \n",b)

## Basic operations
Arithmetic operators on arrays apply element-wise. A new array is created and filled with the result.

In [ ]:
a = np.array([20,30,40,50])
print("a=",a)
# np.arange(): default start value is 0, default step size is 1, 
# so b = [0,1,2,3]
b = np.arange(4)
print("b=", b)
c = a-b
print("c=a-b =", c)
# ** denotes the second power ^2
print("b^2=", b**2) 
print("10 sin(a)=", 10*np.sin(a))
print("a<35:", a<35)

The product operator `*` operates element-wise in NumPy arrays. 
The matrix product can be performed using the `@` operator or the `dot` function:

In [ ]:
# Create two numpy arrays starting from lists of lists
A = np.array( [[1,1],   
               [0,1]] )
B = np.array( [[2,0],
               [3,4]] )
print("A*B= \n", A * B, "\n")                # elementwise product
print("A@B= \n", A @ B, "\n")                # matrix product
print("A.dot(B)=\n", A.dot(B))               # another matrix product

NumPy provides familiar mathematical functions such as sin, cos, and exp. In NumPy, these are called “universal functions”(`ufunc`). Within NumPy, these functions operate elementwise on an array, producing an array as output.

In [ ]:
B = np.arange(3) # B=[0,1,2]
np.exp(B)

# Spectral graph theory
We will explore several notions from spectral graph theory by analysing the following graph:

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

G = nx.DiGraph()
nx.add_cycle(G,[1,2,3,4])
nx.add_cycle(G,[4,3,2,1])

nx.draw_circular(G, with_labels=True)

plt.show()

First we construct the `weight matrix` $W$ (called also `adjacency matrix` in NetworkX), the diagonal matrix $D$, and the normalized adjacency matrix $P$

In [ ]:
W = nx.adjacency_matrix(G) # -> return type is scipy.sparse.csr_matrix
# convert W to a numpy array
W = W.toarray()
print("W:",W, "\n")
degrees = np.sum(W,axis=1)
print("Degrees:",degrees, "\n")
D = np.diag(degrees)
print("D: \n",D,"\n")
# P = D^(-1) W
P = np.linalg.inv(D) @ W
print("P: \n",P)

We compute the powers of the matrix $W$. Powers of $W$ have a useful interpretation in a unweighted (di)graphs, namely, $(W^n)_{ij}$ equals the number of walks of length $n$ from $i$ to $j$.

The interpretation can be extended to weighted graphs.

In [ ]:
# Compute W^2
power = W
power = power @ W
print("W^2 =", power)

In [ ]:
# Compute W^3
power = power @ W
print("W^3 =", power)

# Note that we compute P^3 by using P^2 to save computational time

In [ ]:
# Compute W^11
for n in range(3,11):
    power = power @ W
print("W^11 =", power)

The powers of $W$ still contain zero elements. Why?

The reason is that the graph $G$ is periodic, as can be easily verified. There are no closed walks of odd length.

In [ ]:
print("Is G aperiodic:",nx.is_aperiodic(G))

Let us modify the graph to make it aperiodic, and compute again the powers of $W$.

In [ ]:
G2 = nx.DiGraph()
nx.add_cycle(G2,[1,2,3,4])
nx.add_cycle(G2,[4,3,2,1])
G2.add_edge(1,3)
G2.add_edge(3,1)

print("Is G2 aperiodic:",nx.is_aperiodic(G2))

nx.draw_circular(G2, with_labels=True)

plt.show()

In [ ]:
W2 = nx.adjacency_matrix(G2)
W2 = W2.toarray()

degrees = np.sum(W2,axis=1)
print("Degrees",degrees, "\n")

D2 = np.diag(degrees)
print("D2: \n",D2,"\n")

P2 = np.linalg.inv(D2) @ W2
print("P2: \n",P2,"\n")

**Question**: what is the smallest power of $W$ that does not have null elements?

**Hint**: exploit the fact that in unweighted (di)graphs $(W^n)_{ij}$ is the number of walks between $i$ and $j$.

In [ ]:
N = G2.number_of_nodes()
index = 1

power = W2

while nx.is_aperiodic(G2):
    if np.count_nonzero(power) == N*N:
        print("The smallest power of W with all non-zero elements is:", index)
        break
    else:
        index += 1
        power = power @ W2
        
print(power)

We can also observe that $G$ is bipartite, while $G_2$ is not. To this end, we exploit the notion of **coloring**.

A coloring is a function $\phi$ that assigns to every node $n$ a color $\phi(n)$ with the property that, for every pair of adjacent nodes $\{n,m\}$, then $\phi(n) != \phi(m)$
 
**Theorem**: a graph is bipartite if and only if there exists a 2-coloring (coloring with 2 colors) defined on the graph.

In [ ]:
# We construct a coloring for the graph G
color_map = []

for node in range(G.number_of_nodes()):
    if node % 2 == 1:
        color_map.append('blue')
    else: 
        color_map.append('red')
        
print(color_map)

nx.draw_circular(G, node_color=color_map, with_labels=True)


plt.show()

Since $G$ admits a coloring, $G$ is bipartite.
Note that a 2-coloring for $G_2$ does not exist, thus $G_2$ is not bipartite. Another way to see this, is by the following theorems.

**Theorem 1**: an undirected graph is bipartite if and only if all circuits have even length.

We can also leverage algebraic graph theory. The following result is a consequence of Perron-Frobenius theorem.

**Theorem 2**: if a graph is bipartite, then the spectrum of its normalized adjacency matrix $P$ contains $-1$

In [ ]:
# eig(P) returns the eigenvalues (in vector w)
# and eigenvectors (in matrix v) of P

w,v = np.linalg.eig(P)
print("spectrum of P:",w)

w,v = np.linalg.eig(P2)
print("spectrum of P2:",w)

Theorem 1 states that $G$ is bipartite and $G_2$ is not. Theorem 2 only states that $G_2$ is not bipartite.

The trace of the n-th power of $W$ equals the number of circuits with length $n$ in the graph ($n\ge2$).

**Question**: what is the trace of $W^3$?

To answer the question, let us plot again $G$

In [ ]:
plt.subplot(121)
nx.draw_circular(G, with_labels=True)

plt.subplot(122)
nx.draw_circular(G2, with_labels=True)

plt.show()

In [ ]:
print("Number of length-3 circuits in G:", np.matrix.trace(W@W@W))
print("Number of length-3 circuits in G2:", np.matrix.trace(W2@W2@W2))

We now define a new graph.

In [ ]:
G = nx.DiGraph()
G.add_nodes_from(range(1,11))
nx.add_cycle(G,[1,2,3])
nx.add_cycle(G,[4,5,6])
nx.add_cycle(G,[8,9,10])
G.add_edges_from([(4,3), (3,8), (5,7), (7,7)])

# define pos according to spring layout
# to fix nodes' positions in all graph drawings.
# spring_layout positions nodes using Fruchterman-Reingold force-directed algorithm.
pos = nx.spring_layout(G)
nx.draw(G,pos, with_labels=True)

plt.show()

## Invariant probability distributions
In this section we show how to compute the invariant distributions of a graph.

**Definition**: an `invariant distribution` $\pi$ of a graph is a normalized eigenvector of $P'$ relative to eigenvalue $1$, i.e.,

$$
P' \pi = \pi, \quad \sum_i \pi_i = 1
$$ 
To do this, we first compute all eigenvalues and eigenvectors of $P'$ with function `np.linalg.eig()`:

In [ ]:
W = nx.adjacency_matrix(G)
W = W.toarray()
degrees = np.sum(W,axis=1)
print("Degrees:",degrees,"\n")
D = np.diag(degrees)
print("D: \n",D,"\n")
# P = D^(-1) W
P = np.linalg.inv(D) @ W
print("P: \n",P)

**Theorem**: if the graph is strongly connected, the eigenvalue 1 has multeplicity 1 (only one invariant distribution)

We compute the condensation graph of $G$

In [ ]:
CG = nx.algorithms.components.condensation(G)
nx.draw(CG, with_labels=True)

plt.show()

print(dict(CG.nodes))

The graph is not strongly connected, since the condensation graph has more than one node. 

To confirm this, we compute the spectrum of $P$, and show that the eigenvalue 1 has algebraic multeplicity greater than 1.

**Remark**: multeplicity 1 of eigenvalue 1 does not imply strong connectedness of the graph.

In [ ]:
# eig(P.T) returns the eigenvalues (in vector w)
# and eigenvectors (in matrix v) of P'
w,v = np.linalg.eig(P.T)
print("eigenvalues:",w) # -> the 0th and 5th eigenvalues are 1

There are two eigenvalues equal to 1 (in position 0 and 5 of `w`). We select the eigenvectors corresponding to the two occurrencies of eigenvalue 1 and we normalize them to obtain the two invariant distributions (`pi0` and `pi5`):

In [ ]:
# we iterate over indices corresponding to eigenvalues 1
# i.e. corresponding to entries of w that are equal 1:
# for each index we extract the corresponding eigenvector in v
# and normalize it

# we use np.isclose() to compare eigenvalues to 1 to avoid
# numerical precision errors
for index in [i for i in range(len(G)) if np.isclose(w[i],1)]: 
    pi = v[:,index].real  # -> eigenvectors are complex but invariant distributions are real, so we convert pi to real
    pi = pi/np.sum(pi) # normalization
    print("pi", index, "=", pi)

**Theorem**: the multeplicity of the eigenvalue 1 equals the number of trapping sets of the graph. Moreover, for every trapping set, there exists an invariant distribution (called extremal) whose support coincides with the node set of the trapping set.

**Remark**: the extremal invariant distribution supported on a trapping set may be computed by computing the unique invariant distribution of the subgraph obtained by selecting the nodes of a trapping set.

**Exercise**: Compute the extremal invariant distribution of $G$.

1. Find the trapping sets of the graph
2. For each trapping set, construct the corresponding induced subgraph
3. Compute the P matrix of each induced subgraph and its invariant measure
4. Map the obtained measures back to the original graph G (by adding zeros in the appropriate positions)

**Hint**: use the methods introduced in the previous notebook, and the code above.

In [ ]:
# TO DO

Up to now we have exploited the function `numpy.linalg.eig` to compute all the eigenvalues and eigenvectors of $P'$, and we have selected the leading ones. 

Another approach consists in applying an iterative method which converges to the leading eigenvector. This will be illustrated in the following weeks.

# Network centralities

Node centralities measure the importance of the nodes of the network. Several notions of centrality may be defined, e.g.:

**Degree centrality**: the centrality of a node is proportional to its degree. In digraphs, we can define both the indegree and the outdegree centrality. For instance, the indegree centrality is a measure of importance in Twitter network, since it measures the number of followers of every account.

**Eigenvector centrality**: the eigenvector centrality generalizes the degree centrality. Instead of counting the number of neighbors as the degree centrality does, this centrality gives more importance to connections with more central nodes. Let $z$ denote the centrality. The eigenvector centrality satisfies

$$
z_i \propto \sum_{j} W_{ji}z_j
$$

By normalizing $z$, and taking the proportionality factor equal to dominant eigenvalue $\lambda_W$, we obtain $\lambda_W z = W'z$, i.e., $z$ is the dominant eigenvector of $W$, and has non-negative components because of Perron-Frobenius theorem.

## Small networks Example: Zachary's Karate Club
Zachary's Karate Club network is a well-know network example. This is a quite small network so we can compute centralities directly. To better understand the meaning of centrality measure it is useful to visualize them by producing appropriate graph representations.

Let's first load and visualize Zachary's Karate Club network:

In [ ]:
G = nx.karate_club_graph()

# Fix node positions on all pictures according to spring layout
pos = nx.spring_layout(G) 
nx.draw_networkx(G, pos)

plt.show()

# Visualizing centralities on graphs
We can compute centralities both using NetworkX algorithms and performing iterative procedures. It is important to make sense of the centrality vectors, and a useful way to do this is by visualizing centralities on graphs.

Compute, for example, the degree centrality of $G$. The following code shows how to represent $G$ so that nodes size and color reflects their centrality value:

In [ ]:
# Degree centrality

# dc is a dictionary with nodes as keys and degree centralities as values
dc = nx.degree_centrality(G) 
plt.figure(1, figsize=(10,7))
# we draw the graph with same node position "pos" defined above
nx.draw(G,pos,
         with_labels=True,
         # keys of dc are nodes
         nodelist=dc.keys(), 
         # node size is proportional to centrality value
         node_size = [d*10000 for d in dc.values()], 
         # node's color reflects centrality values (higher dc = darker color)
         node_color=list(dc.values()),
         font_size=8,
         # node's colors are on the red scale
         cmap=plt.cm.Reds) 

plt.show()

We repeat this procedure with a different measure, eigenvector centrality:

In [ ]:
# Eigenvector centrality
ec = nx.eigenvector_centrality(G)
plt.figure(1, figsize=(10,7))
nx.draw(G, pos,
          with_labels=True,
          nodelist=ec.keys(),
          # node size is proportional to eigenvector centrality
          node_size = [d*10000 for d in ec.values()],  
          node_color=list(ec.values()),
          font_size=8,
          cmap=plt.cm.Reds,
          )

plt.show()

It is interesting to compare different centrality measures for the same graph and see how they are correlated. Below we visualize the correlation between degree centrality and eigenvector centrality of $G$:

In [ ]:
# Correlation degree-eigenvector

# x corresponds to degree centrality values
xdata = list(dc.values()) 
# y corresponds to eigenvector centrality values
ydata = list(ec.values()) 

plt.figure(1, figsize=(7,7))
# for each node, we plot an "+" with coordinates equal to the values of its
# degree and eigenvector centrality
plt.plot(xdata,ydata, '+') 
plt.xlabel('Degree Centrality')
plt.ylabel('Eigenvector Centrality')

plt.show()

The two centralities appear to be correlated for this network. To explore this in more details it is useful to add node ids, so that we can see which are the nodes with higher or lower correlation.

In [ ]:
# Adding node ids:

# We define a figure and we construct two subplots: 
# on the left we plot the centralities correlation diagram
# with node labels, on the right we draw the graph 
# with same node labels
fig = plt.figure(1, figsize=(14,7))
# add_subplot() returns the axes of the subplot
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

for v in range(len(G)):
    # Axes.text(x,y,s) add the text s to Axes instance (i.e., to the subplot)
    # at location x, y. For each node v we plot 
    # node ids in position (xdata[v], ydata[v]) where xdata = list(dc.values())
    # and ydata = list(ec.values())
    ax1.text(x = xdata[v], y = ydata[v], s=str(v))
# we set the limits for x and y scales

ax1.set_xlim(0, 0.6)
ax1.set_ylim(0, 0.4)
ax1.set_xlabel('Degree Centrality')
ax1.set_ylabel('Eigenvector Centrality')

nx.draw_networkx(G, pos, ax=ax2)

plt.show()

Observe that node 11 has degree 1, while node 16 has degree 2, so $dc(16)>dc(11)$.

However, node 11 is connected to node 0, which has a large value of eigenvector centrality, while node 16 is connected to nodes 5 and 6, which have smaller values of centrality. For this reason, $ec(16)<ec(11)$.

**Invariant distribution centrality**: it generalizes the eigenvector centrality by taking into account that being connected to nodes that connect to many nodes is less important than being connected to nodes that connect with a few nodes.

$$
z = P'z
$$

**Katz centrality**: it generalizes the eigenvector centrality by assuming that nodes have also an intrinsic centrality. The centrality is the sum of the intrinsic centrality and the centrality given by the network.

$$
z =  \frac{1-\beta}{\lambda_W} W' z + \beta \mu\,, \quad \beta \in [0,1].
$$

If $\beta = 0$, the Katz centrality coincides with the eigenvector centrality. If $\beta = 1$, then $z=\mu$.

**Bonacich centrality (or Page-rank)**: it generalizes the invariant distribution centrality by assuming that nodes have also an intrinsic centrality. The centrality is the sum of the intrinsic centrality and the centrality given by the network.
$$ 
z = (1-\beta)P' z + \beta \mu\,, \quad \beta \in [0,1].
$$

If $\beta = 0$, it is the invariant distribution centrality. If $\beta = 1$, then $z=\mu$.

There are two ways to compute Katz and Bonacich centralities: **direct** and **iterative**.
We start by computing those centralities by direct methods for the Zachary's karate club graph.

## Direct method (for didactic purposes)
Direct methods consist in inverting the equation above and computing directly the centrality. Notice that

1. the Katz centrality 
$ z =  (\mathbf{I}-\frac{1-\beta}{\lambda_W} W')^{-1} \beta \mu $

2. and Bonacich centrality 
$ z = (\mathbf{I}-(1-\beta)P')^{-1} \beta \mu $

Note that the inversion can be done (if $\beta > 0$) because the matrices $\frac{1-\beta}{\lambda_W} W'$ and $(1-\beta)P'$ have spectral radius less than 1


In [ ]:
# compute matrices of the graph
W = nx.adjacency_matrix(G)
W = W.toarray()
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

In [ ]:
N = G.number_of_nodes() 
beta = 0.15
mu = np.ones((N,1))
# note that the normalization of mu does not influence z, if we consider normalized centralities

# compute the largest eigenvalue of W (which is real because of Perron-Frobenius theorem)
w,v = np.linalg.eig(W)
w = w.real

lambda_max = max(w) 
zk = np.linalg.inv(np.diag(np.ones(N)) - W.T*(1-beta)/lambda_max) * beta @ mu
# normalize the centrality
zk = zk/sum(zk)

print(zk)

### Bonacich (Page-rank) centrality
Page-rank centrality is the Bonacich centrality with $\mu=\mathbf{1}$ and $\beta=0.15$.

In [ ]:
zb = np.linalg.inv(np.diag(np.ones(N)) - P.T*(1-beta)) * beta @ mu
zb = zb/sum(zb)

# transform centralities to float
val = [];
for i in zb:
    val.append(float(i))

zb_list = val

# sometimes it is useful to store the centralities in a dictionary
# create a dictionary to collect the centralities, with nodes as keys and their centrality as values
zip_iterator = zip(G.nodes(), zb_list)
zb_dict = dict(zip_iterator)

print(zb_dict)

## Compute centralities by networkX functions
The function `algorithms.link_analysis.pagerank_alg.pagerank` computes the Page-rank centrality of a given network

In [ ]:
zb2_dict = nx.algorithms.link_analysis.pagerank_alg.pagerank(G)

# check if the centrality are normalized
zb2 = np.array(list(zb2_dict.values()))
print("Normalization:", sum(zb2), "\n")

# transform values to float
zb2_list = [];
for i in zb2:
    zb2_list.append(float(i))

Compare the Page-rank centrality computed by the inversion formula with the one computed by NetworkX

In [ ]:
# for comparison, it is convenient to use centralities as arrays
# before comparing we ensure that the shape of the arrays is the same

print("Shape of zb:", zb.shape)
print("Shape of zb2", zb2.shape)

In [ ]:
# since it is not, we reshape zb
zb = zb.reshape(N)

# now we can compute the distance
print("Distance between zb and zb2:", np.linalg.norm(zb-zb2))

# Iterative methods
The smartest way to compute Bonacich centrality (or Katz centrality) is by iterative methods.

Consider the dynamics
$$
\begin{cases} 
z(t+1) = (1-\beta)P'z(t) + \beta \mu  \\
z(0) = z_0
\end{cases}
$$

The transient of the dynamics is

$$
\begin{cases}
z(1) = (1-\beta)P'z(0) + \beta \mu\\
z(2) = (1-\beta)^2 (P')^2 z(0) + (1-\beta)P' \beta \mu + \beta \mu\\
\vdots\\
z(t) = (1-\beta)^t (P')^t z(0) + \sum_{i=0}^{t-1} (1-\beta)^i (P')^i \beta \mu
\end{cases}
$$

The first term vanishes as $t \to +\infty$ because $(1-\beta)P'$ is sub-stochastic, while the second term is a geometric sum. The dynamics thus converges to the limit

$$
\lim_{t \to +\infty} z(t) = (\mathbf{I}-(1-\beta) P')^{-1} \beta \mu,
$$

which is the Bonacich centrality of the graph.

**Remark 1**: You should never use direct ways to compute centralities if iterative algorithms are available. The iterative method is more efficient than the direct one as the order of the graph grows, since it does not involve the inversion of a $N \times N$ matrix.

**Remark 2**: Note that the convergence of $z(t)$ to the Bonacich centrality holds for every initial condition $z_0$ and the limit is independent of $z_0$.

**Remark 3**: Notice that the proposed method is **distributed**, i.e., the operations at single node levels do not require a complete knowledge of the network. Each node $i$ updates its state $z_i(t+1)$ by using only local information, i.e., the i-th row of $W$ and the state $z_j(t)$ of nodes $j$ that are adjacent to $j$.

Let us implement the distributed method.

In [ ]:
N = G.number_of_nodes()
beta = 0.15
mu = np.ones((N,1))/N

# arbitrary initial condition: 1/N-uniform vector of size N (initial condition does not matters)
z_0 = np.ones((N,1))/N

# set a tolerance to assess convergence to the limit
tol = 1e-5

# run the dynamics
z_old = z_0

while True:
    z_new = P.T @ z_old * (1-beta) + beta * mu
    if np.linalg.norm(z_new-z_old) < tol:
        break
    z_old=z_new

zb_distr = z_new

# normalize the centrality
zb_distr = zb_distr / sum(zb_distr)

print("Bonacich centrality: \n", zb_distr)

## Katz centrality

For Katz centrality, consider the following dynamics:

$$
\begin{cases} 
z(t+1) = \frac{(1-\beta)}{\lambda_W}W'z(t) + \beta \mu  \\
z(0) = z_0.
\end{cases}
$$

It is easy to prove that the dynamics converges to the Katz centrality.

**Remark**: notice that the proposed method is iterative (and therefore more efficient than direct method), but it is not distributed. Indeed, to perform the computation, every node needs to know $\lambda_W$, which is a global information on the network.

## Exercise
Compute the Katz centrality of the Zachary's karate club graph by iterative methods, and compare the results with direct methods.

**Hint**: use the definition of Katz centrality and same techniques as above

In [ ]:
# TO DO

# Testing sensitivity of measures
In this section we will check the dependence of centrality measures with respect to their paramenters and the sensitivity of the iterative algorithms to compute such measure with respect to the number of iterations.

## The effect of parameters
In our first experiment we analyze the dependence of Page Rank centrality on the parameter $\alpha=1-\beta$. We set distinct values for $\alpha$ while we fix the number of iterations, and run Page Rank. Then we plot the resulting Page Rank values with respect to $\alpha$.

In [ ]:
fig = plt.figure(2, figsize=(16,7))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)

pos = nx.spring_layout(G) 
nx.draw_networkx(G, pos, ax=ax1)

# we consider values for alpha from 0 to 1 with step size 0.25
alphas = np.arange(0, 1.25, 0.25)

for alp in alphas:
    # pagerank has parameters alpha and mu:
    # note that alpha = 1-beta and weight parameters mu are set to 1 by default
    pr = nx.pagerank(G, alpha=alp) 
    prval = list(pr.values())
    ax2.plot(prval, color=np.random.rand(3), label='alpha {0:.2f}'.format(alp))
    
ax2.legend()

plt.show()

Let's explain the previous result. Keep in mind that the parameter alpha used by `nx.pagerank` corresponds to $1-\beta$, and the centrality $z$ satisfies

$$
z = (1-\beta)P'z + \beta \mu
$$

- As $\alpha = 0$ ($\beta = 1$), $z = \mu$, i.e., Bonacich is equivalent to the intrinsic centrality $\mu$, the network does not play any role.

- As $\alpha = 1$ ($\beta = 0$), $z = P'z$, i.e., Bonacich is equivalent to invariant distribution centrality (the intrinsic centrality is irrelevant).

In between these two extreme cases there is a combination of network effects and intrinsic centrality.

## Exercise
Repeat the analysis. This time keep $\alpha$ fixed to 0.5 and select 3 different non-uniform vectors $\mu$ as `personalization` parameter to `pagerank`. How do you interpret the result?

## The effect of iteration number
In this section we consider a bigger network and we analyse the speed of convergence of iterative algorithms for computing centrality measures. 
 

In [ ]:
G = nx.read_gml('polblogs.gml')
print("Type of G:", type(G))

Since $G$ is a multidigraph, we define an equivalent digraph to compute the centralities (the function 'pagerank' does not work with multigraphs)

In [ ]:
GG = nx.DiGraph()
for n, nbrs in G.adjacency():
    # edict is a dictionary of dictionaries; 
    # the keys of edict are parallel edges from n to nbr;
    # the values of edict are dictionary,
    # containing attribute values of the corresponding edge
    for nbr, edict in nbrs.items(): 
        # each edge has weight=1, so total value is just  
        # the number of parallel edges
        total_value = len(edict) 
        GG.add_edge(n, nbr, weight = total_value)

The graph is very large, thus we cannot plot it.

In [ ]:
print(GG.number_of_nodes())

We now test the convergence speed of `nx.pagerank` algorithm:

In [ ]:
fig = plt.figure(1, figsize=(16,7))

# set 3 iteration numbers
iters = [10,15,50]
# define the position of the next plot in the subplot grid
position = 1
# create a list to collect the page rank values obtained in the three runs 
prvals = []

for max_iter in iters:
    # compute page rank
    pr = nx.pagerank(GG, max_iter = max_iter) 
    # compute page rank values
    prval = list(pr.values())
    # append the result to the list
    prvals.append(np.array(prval)) 
    # create a new sublot in the grid
    ax = fig.add_subplot(2,2,position)
    # plot the PR values
    ax.plot(prval, color=np.random.rand(3), label='{0:d} iterations'.format(max_iter))
    position+=1

# add a legend which contains all labels
# informations specified in previous plot calls
fig.legend()  
# we assume the values obtained with nx.pagerank()
# with no iterations constraints as a benchmark
benchmark = np.array(list(nx.pagerank(GG).values())) 
# we compute errors as norm of the differences wrt the benchmark
errors = [np.linalg.norm(prval-benchmark) for prval in prvals]
print("Errors:", errors)


plt.show()

`nx.pagerank` algorithm converges very fast, in 10 iterations! 

### Exercise
Check if our iterative algorithm for computing Bonacich centrality is as good as this by performing a similar analysis.

In [ ]:
# TO DO

## An interpretation of Katz (and Bonacich) centrality
Bonacich centrality (as well as Katz centrality) can be interpreted in terms of walks on the graphs.
We show this by an example.

We shall make use of an undirected graph and Katz centrality to simplify the analysis.

In [ ]:
G = nx.lollipop_graph(6,3)
G.add_edges_from([(9,8),(10,8),(11,8),(12,8),(13,8)])

pos = nx.spring_layout(G) 
nx.draw(G, pos, with_labels = True)

N = len(G)

# compute matrices of the graph
W = nx.adjacency_matrix(G)
W = W.toarray()
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

# compute the largest eigenvalue of W
w,v = np.linalg.eig(W)
w = w.real

lambda_max = max(w) 

We consider the iterative implementation of Katz centrality
$$
\begin{cases} 
z(t+1) = \frac{(1-\beta)}{\lambda_W}W'z(t) + \beta \mu  \\
z(0) = z_0.
\end{cases}
$$
with uniform $z_0$ and $\mu$.

In [ ]:
mu = np.ones((N,1))/N
beta = 0.15
# initial centrality distribution
z = np.ones((N,1))/N
z_reshape = z.reshape(N)

print("Centralities at iteration 0:", z_reshape)

Note that normalizing $\mu$ does not modify the centrality, since in

$$ 
z =  (\mathbf{I}-\frac{1-\beta}{\lambda_W} W')^{-1} \beta \mu 
$$

$\mu$ affects the normalization of $z$ only (same for Bonacich centrality). 

However, the normalization of $\mu$ affects the transient of the iteration to compute $z$. In particular, if one considers the **Bonacich centrality**, using $\mu, z_0$ such that $\mathbf{1}' \mu = \mathbf{1}' z_0 = \mathbf{1}$ is preferable, since it guarantees that, if $\mathbf{1}' z(0)=1$, then $\mathbf{1}' z(t)=1$ for every $t$. Indeed, if $\mathbf{1}' z(t-1)= 1$, then 

$$
\mathbf{1}' z(t) = (1-\beta) \mathbf{1}' P' z(t-1) + \beta \mathbf{1}'\mu = (1-\beta) \mathbf{1}' z(t-1) + \beta = (1-\beta) + \beta = 1
$$

### Back to Katz centrality

After 1 iteration,
$$
z(1) = \frac{(1-\beta)}{\lambda_W}W'z(0) + \beta \mu,
$$

which means that the centrality of a node is the a combination of its intrinsic centrality, and the centrality of the neighbors. Since the intrinsic centrality is the same for every node, if we start with a uniform $z(0)$, $z(1)$ depends only on the degree of the node.

Similar observation can be made for the Bonacich centrality.

In [ ]:
# after 1 iteration
z = W.T @ z * (1-beta)/lambda_max + beta * mu

z_reshape = z.reshape(N)

nodesize=z_reshape*7000

# plot centrality at iteration 0
plt.figure(1, figsize=(10,7))
# we draw the graph with same node position "pos" defined above
nx.draw(G,pos,
         with_labels=True,
         nodelist=list(G.nodes()), 
         # node size is proportional to centrality value
         node_size = nodesize, 
         # node's color reflects centrality values (higher dc = darker color)
         node_color=z_reshape,
         font_size=8,
         # node's colors are on the red scale
         cmap=plt.cm.Reds) 

print("Centralities at iteration 1:", z_reshape)


plt.show()

As the number of iterations grows, $z(n)$ takes into account also nodes at greater distance.

At the equilibrium, the centrality $z^*$ can be interpreted in terms of walks as follows:

\begin{equation}
\begin{aligned}
z^*&=\lim_{n \to +\infty}z(n)=\sum_{n = 0}^{\infty} \left(\frac{(1-\beta)}{\lambda_W}\right)^n (W')^n \beta \mu \\
   &= \beta \mu + \frac{(1-\beta)}{\lambda_W} (W') \beta \mu + \left(\frac{(1-\beta)}{\lambda_W}\right)^2 (W')^2 \beta \mu + \cdots
\end{aligned}
\end{equation}

If one considers node $i$

$$
z^*_i = \beta \mu_i + \frac{(1-\beta)}{\lambda_W}\beta \sum_{j} (W')_{ij} \mu_j + \left(\frac{(1-\beta)}{\lambda_W}\right)^2 \beta \sum_{j} ((W')^2)_{ij} \mu_j + \cdots
$$

**Interpretation**. Since $((W')^n)_{ij}$ is the number of paths of length $n$ from $j$ to $i$, the centrality of node $i$ is the sum of:
- its intrinsic centrality $\mu_i$, plus 
- the intrinsic centrality of its in-neighbors, i.e., $\sum_j W_{ji} \mu_j$, plus 
- the intrinsic centrality of the nodes connected by paths of length 2, and so on... 

Longer paths have a decreasing weight due to the term $(1-\beta)^n$.

**Question**: which node do you expect to have a higher Katz centrality? Why?

In [ ]:
# set a tolerance to assess convergence to the limit
tol = 1e-5

z_0 = np.ones(N,)/N
mu = np.ones(N,)/N

# run the dynamics
z_old = z_0
print()
while True:
    z_new = W.T @ z_old * (1-beta)/lambda_max + beta * mu
    if np.linalg.norm(z_new-z_old) < tol:
        break
    z_old=z_new

zk = z_new

# normalize the centrality
zk = zk/sum(zk)
zk = zk.reshape(N)

print(zk)

plt.figure(1, figsize=(10,7))
# we draw the graph with same node position "pos" defined above
nx.draw(G,pos,
         with_labels=True,
         nodelist=list(G.nodes()), 
         # node size is proportional to centrality value
         node_size = zk*7000, 
         # node's color reflects centrality values (higher dc = darker color)
         node_color=zk,
         font_size=8,
         # node's colors are on the red scale
         cmap=plt.cm.Reds) 


plt.show()

**Question**: how do you expect to be modified the centralities when using Bonacich instead of Katz? Focus on node 8 vs node 5.

**Hint**: recall the definition of the two centralities, i.e.,

- Katz: $z =  \frac{1-\beta}{\lambda_W} W' z + \beta \mu$
- Bonacich: $x = (1-\beta)P' x + \beta \mu$ 

In [ ]:
zb_dict = nx.algorithms.link_analysis.pagerank_alg.pagerank(G)

# check if the centrality are normalized
zb = np.array(list(zb_dict.values()))

print(zk)

plt.figure(1, figsize=(10,7))
# we draw the graph with same node position "pos" defined above
nx.draw(G,pos,
         with_labels=True,
         nodelist=list(G.nodes()), 
         # node size is proportional to centrality value
         node_size = zb*7000, 
         # node's color reflects centrality values (higher dc = darker color)
         node_color=zb,
         font_size=8,
         # node's colors are on the red scale
         cmap=plt.cm.Reds) 


plt.show()

## Exercise

Compute all the centralities for the this graph, plot them, and comment the results.

**Hint**: use the code introduced in the lectures to compute the degree, eigenvector, Katz and Bonacich centralities. For the invariant distribution centrality use the function `np.linalg.eig()` to find the invariant distribution of $P$. Use the code introduced in the last lecture to plot the centralities.

In [ ]:
# TO DO

In [ ]:
attr_components = tuple(nx.algorithms.components.attracting_components(G))

for c in attr_components:
    # construct the induced subgraph with nodes from the attractive component c
    sG = G.subgraph(c)
    # construct the matrix P on the subgraph
    W = nx.adjacency_matrix(sG)
    W = W.toarray()
    degrees = np.sum(W,axis=1)
    D = np.diag(degrees)
    P = np.linalg.inv(D) @ W
    # find the extremal dominant eigenvector corresponding to component c
    w,v = np.linalg.eig(P.T)
    for index in [i for i in range(len(sG)) if np.isclose(w[i],1)]: 
        pi = v[:,index].real  # -> eigenvectors are complex but pi is real, so we convert it to real
        pi = pi/np.sum(pi)
    # map pi back in the original node space
    pi_G = np.zeros(len(G))
    for i in range(len(sG)):
        pi_G[list(sG.nodes)[i]-1] = pi[i] # shift by -1 because nodes are (1,...10) while vector indexes are (0,...,9)
    print("pi:", pi_G, "\n")
    